In [41]:
import csv

def extract_first_column(csv_file):
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader, None)  # skip the headers
        first_column = [row[0] for row in reader]
        return first_column

def print_comma_separated(data):
    if len(data) > 0:
        print(data[0], end='')  # print the first element without comma
        for item in data[1:]:
            print(f', {item}', end='')  # print remaining elements

# Replace 'input.csv' with the path to your CSV file
csv_file = 'data\miRPathDB - 2023-07-01T183931.121.csv'

first_column_data = extract_first_column(csv_file)
print_comma_separated(first_column_data)


A1CF, A2ML1, AADACL2, AADAT, AAK1, AASDH, ABCA12, ABCA6, ABCA9, ABCB11, ABCC1, ABCC5, ABCD4, ABCF1, ABCF3, ABCG2, ABHD15, ABHD17A, ABHD18, ABHD2, ABHD5, ABHD6, ABITRAM, ABL1, ABL2, ABLIM1, ABLIM3, ABRACL, ABRAXAS2, ACAD10, ACADSB, ACBD7, ACKR4, ACO1, ACOD1, ACOT11, ACOT13, ACOT2, ACOX1, ACOX3, ACP1, ACPP, ACSF3, ACSL3, ACSL4, ACSL6, ACSM2A, ACSM2B, ACSM4, ACSS3, ACTC1, ACTN2, ACTR1A, ACTR3, ACTR3B, ACTR5, ACTR8, ACTRT3, ACVR1C, ACVR2B, ACVRL1, ACYP2, ADAM22, ADAM23, ADAM28, ADAM30, ADAM7, ADAMDEC1, ADAMTS1, ADAMTS12, ADAMTS19, ADAMTS2, ADAMTS20, ADAMTS3, ADAMTS4, ADAMTS5, ADAMTSL3, ADAP1, ADAT1, ADAT2, ADCK2, ADCY1, ADCY3, ADCY8, ADCY9, ADCYAP1, ADCYAP1R1, ADD2, ADGRA3, ADGRD2, ADGRE2, ADGRF1, ADGRF5, ADGRG3, ADGRG7, ADGRL1, ADGRL2, ADGRL3, ADH1B, ADH4, ADH5, ADI1, ADIPOR2, ADM2, ADNP, ADRB1, ADSS, AEBP2, AEN, AFAP1L1, AFAP1L2, AFDN, AFF1, AFF2, AFF3, AFM, AGAP1, AGBL4, AGBL5, AGFG1, AGFG2, AGGF1, AGMAT, AGMO, AGO1, AGO3, AGO4, AGPAT3, AGPAT4, AGPAT5, AGPS, AGT, AGTPBP1, AGTRAP, AGXT2,

In [42]:
# Script that generates the data, takes in a two column list, miRNA and gene Targets where gene targets is foramtted like (gene1, gene2, gene3) This also one hot encodes it
import pandas as pd

# Read your csv file into a pandas DataFrame
df = pd.read_csv('data\Cancer.csv')

# Split the 'GeneTarget' column into separate rows
df = df.assign(GeneTarget=df['GeneTarget'].str.split(', ')).explode('GeneTarget')

# Pivot the dataframe to have miRNAs as index, gene targets as columns and fill non-existing pairs with 0
pivot_df = df.groupby(['miRNA', 'GeneTarget']).size().unstack(fill_value=0)

# Convert counts to binary values (0 and 1)
one_hot_df = pivot_df.applymap(lambda x: 1 if x > 0 else 0)

# Write the one-hot encoded DataFrame to a new csv file
one_hot_df.to_csv('data\CancerCombined.csv')


In [15]:
import pandas as pd

def add_missing_columns(df, column_list):
    """
    This function checks if the columns from 'column_list' exist in the dataframe 'df'.
    If a column does not exist, it creates the column and fills it with zeros.

    Parameters:
    df (pandas.DataFrame): the input dataframe
    column_list (list): the list of columns to check/add
    
    Returns:
    pandas.DataFrame: the modified dataframe
    """
    for column in column_list:
        if column not in df.columns:
            df[column] = 0
    return df

# The list of columns to be checked/added
columns = ['CHEK2', 'MOSMO', 'ADRB2', 'B4GALT4', 'CCNA2', 'C15orf62', 'EXOC3-AS1', 'DEF6', 'IL1RAPL1', 'LYPLA1', 'MAPK8', 'HCN1', 'LIPG', 'NPFFR2', 'HRH2', 'PLSCR4', 'MAP9', 'BORCS7', 'MAP3K8', 'MAP3K20', 'BCL10', 'MAP3K7', 'P2RX2', 'GAK', 'MAPK1IP1L', 'ECI1', 'C2orf80', 'KIDINS220', 'EGR3', 'DUSP4', 'GPAM', 'BMP6', 'CDK11B', 'OR2L13', 'ANKRD33', 'CARD18', 'CYTIP', 'CCR7', 'COMTD1', 'COX10', 'RHOV', 'EIF4E', 'EZH2', 'PDS5B', 'DCUN1D4', 'CEP162', 'LRP6', 'NEFM', 'ABHD11', 'POLR2M', 'C1orf131', 'CEP55', 'PALLD', 'B3GLCT', 'DNASE1', 'HTR1E', 'CCDC189', 'MS4A7', 'AXIN2', 'KIF3A', 'DLL1']

# Assume df is your input DataFrame
df = pd.read_csv("unpreppeddiseaseset.csv") # uncomment this line to read a CSV file

df = add_missing_columns(df, columns)
df.to_csv('PreppedOtherDiseaseset.csv')


In [ ]:
# Parses miRNA precursor information from miRbase to get csv of data
import csv
import gzip

# Read the hairpin.fa.gz file and write the extracted data to a CSV file
with gzip.open('hairpin (2).fa.gz', 'rt') as infile, open('miRNA_data.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(["miRNA_ID", "Description", "Sequence"])  # write header

    miRNA_ID = ''
    description = ''
    sequence = ''

    for line in infile:
        if line.startswith('>'):
            if miRNA_ID:  # this avoids writing empty data for the first line
                writer.writerow([miRNA_ID, description, sequence])
            
            parts = line[1:].split()  # split the line into parts
            miRNA_ID = parts[0]
            description = ' '.join(parts[1:])
            sequence = ''  # reset the sequence
        else:
            sequence += line.strip()

    # write the last record
    if miRNA_ID:
        writer.writerow([miRNA_ID, description, sequence])


In [2]:
# New one hot encode script
import pandas as pd

# Read your csv file into a pandas DataFrame
df = pd.read_csv('Parkinsons Data pt 2.csv')

# Split the 'GeneTarget' column into separate rows
df = df.assign(GeneTarget=df['GeneTarget'].str.split(', ')).explode('GeneTarget')

# Pivot the dataframe to have miRNAs and class as index, gene targets as columns and fill non-existing pairs with 0
pivot_df = df.groupby(['MiRNA', 'Class', 'GeneTarget']).size().unstack(fill_value=0)

# Convert counts to binary values (0 and 1)
one_hot_df = pivot_df.applymap(lambda x: 1 if x > 0 else 0)

# Reset the index
one_hot_df.reset_index(inplace=True)

# Write the one-hot encoded DataFrame to a new csv file
one_hot_df.to_csv('your_output_file.csv', index=False)


In [29]:
import pandas as pd
import random

# Assume the csv file is named 'miRPathDB (92).csv' and has a column named 'miRNA'
data = pd.read_csv('mirpathdbtotaldata.csv')

# Filter the DataFrame to only include miRNAs that start with 'hsa'
hsa_data = data[data['miRNA'].str.startswith('hsa')]

# Randomly select 40 miRNAs
random_miRNAs = hsa_data['miRNA'].sample(40)

# Print the random miRNAs
print(random_miRNAs)


19444         hsa-miR-6071
3360       hsa-miR-7855-5p
13009      hsa-miR-24-1-5p
18916      hsa-miR-4712-3p
18325        hsa-miR-12121
25310      hsa-miR-6807-5p
18781      hsa-miR-6500-3p
142         hsa-miR-140-5p
16928         hsa-miR-8076
12580         hsa-miR-5092
23031         hsa-miR-3670
22752         hsa-miR-4431
5960          hsa-miR-4420
2789       hsa-miR-130a-5p
17294         hsa-miR-6086
430         hsa-miR-545-3p
25435      hsa-miR-3173-3p
19833         hsa-miR-4451
6695           hsa-miR-639
12290       hsa-miR-489-5p
6380           hsa-miR-941
20121         hsa-miR-4767
5275       hsa-miR-6838-5p
86          hsa-miR-150-5p
5142          hsa-miR-5696
8394       hsa-miR-517b-3p
21240       hsa-miR-432-3p
8           hsa-miR-144-3p
14690     hsa-miR-548ap-3p
21681      hsa-miR-4714-3p
18329        hsa-miR-12125
9833     hsa-miR-103a-1-5p
267         hsa-miR-126-5p
17246         hsa-miR-4534
11866         hsa-miR-4718
12243      hsa-miR-6829-3p
7          hsa-miR-199a-5p
2

In [5]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Assuming you have a CSV file named 'data.csv'
df = pd.read_csv('old data\Raw Data Final2.csv')

# Split the comma separated gene targets into a list
df['GeneTarget'] = df['GeneTarget'].str.split(',')

# Use MultiLabelBinarizer to convert gene targets to columns
mlb = MultiLabelBinarizer()
expandedLabelData = mlb.fit_transform(df['GeneTarget'])
labelClasses = mlb.classes_

# Create a new DataFrame from our expanded label data
expandedLabels = pd.DataFrame(expandedLabelData, columns=labelClasses)

# Concatenate our original DataFrame with our new `expandedLabels` DataFrame
df = pd.concat([df, expandedLabels], axis=1)

# Drop the original 'gene_targets' column (optional)
df = df.drop('GeneTarget', axis=1)

# Now df should be in the format you want
df.to_csv('your_output_file.csv', index=False)


In [43]:
import pandas as pd

# List of columns you want to select
columns = ['CHEK2', 'MOSMO', 'ADRB2', 'B4GALT4', 'CCNA2', 'C15orf62', 'EXOC3-AS1', 'DEF6', 'IL1RAPL1', 'LYPLA1', 'MAPK8', 'HCN1', 'LIPG', 'NPFFR2', 'HRH2', 'PLSCR4', 'MAP9', 'BORCS7', 'MAP3K8', 'MAP3K20', 'BCL10', 'MAP3K7', 'P2RX2', 'GAK', 'MAPK1IP1L', 'ECI1', 'C2orf80', 'KIDINS220', 'EGR3', 'DUSP4', 'GPAM', 'BMP6', 'CDK11B', 'OR2L13', 'ANKRD33', 'CARD18', 'CYTIP', 'CCR7', 'COMTD1', 'COX10', 'RHOV', 'EIF4E', 'EZH2', 'PDS5B', 'DCUN1D4', 'CEP162', 'LRP6', 'NEFM', 'ABHD11', 'POLR2M', 'C1orf131', 'CEP55', 'PALLD', 'B3GLCT', 'DNASE1', 'HTR1E', 'CCDC189', 'MS4A7', 'AXIN2', 'KIF3A', 'DLL1']

# Load your dataset
df = pd.read_csv('data\CancerCombined.csv')

# Add missing columns with NaN values
for col in columns:
    if col not in df.columns:
        df[col] = pd.np.nan

# Select the columns
df = df[columns]

# Save the new dataframe to a csv file
df.to_csv('FinishedPreppedCancer.csv', index=False)


C:\Users\Alex Kumar\AppData\Local\Temp\ipykernel_55568\2403098148.py:12: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df[col] = pd.np.nan
C:\Users\Alex Kumar\AppData\Local\Temp\ipykernel_55568\2403098148.py:12: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df[col] = pd.np.nan
C:\Users\Alex Kumar\AppData\Local\Temp\ipykernel_55568\2403098148.py:12: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df[col] = pd.np.nan
C:\Users\Alex Kumar\AppData\Local\Temp\ipykernel_55568\2403098148.py:12: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df[col] = pd.np.nan
C:\Users\Alex Kumar\AppData\Local\Temp\ipykernel_55568\2403098148.py:12: FutureWarni